**LSB STEGANOGRAPHY EMBEDDING**

In [14]:
!pip -q install pillow
from PIL import Image
import numpy as np

def _text_to_bits(text):
    bits = []
    for b in text.encode('utf-8'):
        bits.extend(f'{b:08b}')
    return bits

def _int_to_32bits(n):
    return list(f'{n:032b}')

def encode_lsb(cover_path, secret_text, output_path='stego_output.png'):
    img = Image.open(cover_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    arr = np.array(img)
    h, w, c = arr.shape
    capacity_bits = h * w * c

    msg_bytes = secret_text.encode('utf-8')
    msg_len = len(msg_bytes)
    msg_bits = _text_to_bits(secret_text)
    header_bits = _int_to_32bits(msg_len)
    payload_bits = header_bits + msg_bits

    if len(payload_bits) > capacity_bits:
        raise ValueError(f"Message too large! Capacity = {capacity_bits} bits, Required = {len(payload_bits)} bits")

    flat = arr.flatten()
    for i, bit in enumerate(payload_bits):
        flat[i] = np.uint8((int(flat[i]) & ~1) | int(bit))

    stego_arr = flat.reshape(arr.shape)
    stego_img = Image.fromarray(stego_arr.astype(np.uint8))
    stego_img.save(output_path, format='PNG')

    print(f"Stego image created successfully!")
    print(f"Saved as: {output_path}")
    print(f"Hidden message length: {msg_len} bytes")

cover_path = "cover.jpeg"
secret_message = "This is CrypticCanvas secret message"
output_path = "stego_output.png"

encode_lsb(cover_path, secret_message, output_path)


Stego image created successfully!
Saved as: stego_output.png
Hidden message length: 36 bytes


**LSB STEGANOGRAPHY DETECTION**

In [15]:
!pip -q install pillow
from PIL import Image
import numpy as np

def _bits_from_array(arr):
    flat = arr.flatten()
    return [str(int(x) & 1) for x in flat]

def _bits_to_text(bits):
    out = bytearray()
    for i in range(0, len(bits), 8):
        out.append(int(''.join(bits[i:i+8]), 2))
    return out.decode('utf-8', errors='replace')

def decode_lsb(stego_path):
    img = Image.open(stego_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    arr = np.array(img)
    bits = _bits_from_array(arr)

    header = ''.join(bits[:32])
    msg_len = int(header, 2)
    msg_bits = bits[32:32 + msg_len * 8]
    message = _bits_to_text(msg_bits)
    return message

stego_path = "stego_output.png"
decoded_message = decode_lsb(stego_path)
print("\n Extracted Hidden Message:")
print(decoded_message)



 Extracted Hidden Message:
This is CrypticCanvas secret message
